<a href="https://www.kaggle.com/code/jtkhande/go-daddy-forecasting-arima?scriptVersionId=122534133" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

#### Autoregressive integrated moving average (ARIMA)
*This model is a commonly used method for time series forecasting*
*The difference between ARIMA and ARMA model's is that the time series is non stationary in ARIMA*

## Dependencies

In [2]:
#Built Directly on Kaggle Note book
#The container already has pre installed packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
from tqdm.notebook import tqdm
from IPython.display import display
import warnings
import copy

### Removing all warnings

In [3]:
warnings.filterwarnings("ignore")


## Loading Datasets

In [4]:
train = pd.read_csv("/kaggle/input/godaddy-microbusiness-density-forecasting/train.csv")
test = pd.read_csv("/kaggle/input/godaddy-microbusiness-density-forecasting/test.csv")
census_starter = pd.read_csv("/kaggle/input/godaddy-microbusiness-density-forecasting/census_starter.csv")
revealed_test = pd.read_csv("/kaggle/input/godaddy-microbusiness-density-forecasting/revealed_test.csv")

### Display Data

In [5]:
print('census_starter df:')
display(census_starter.head())

print('\n','train df:','rows:', len(train))
display(train.head())

print('\n','test df:', 'rows:', len(test))
display(test.head())

print('\n','revealed_test df:', 'rows:', len(revealed_test))
display(revealed_test.head())

census_starter df:


,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,76.6,78.9,80.6,82.7,85.5,1001,14.5,15.9,16.1,16.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,74.5,78.1,81.8,85.1,87.9,1003,20.4,20.7,21.0,20.2,...,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0
2,57.2,60.4,60.5,64.6,64.6,1005,7.6,7.8,7.6,7.3,...,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0
3,62.0,66.1,69.2,76.1,74.6,1007,8.1,7.6,6.5,7.4,...,1.2,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0
4,65.8,68.5,73.0,79.6,81.0,1009,8.7,8.1,8.6,8.9,...,1.3,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0



 train df: rows: 122265


,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243



 test df: rows: 25080


,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01



 revealed_test df: rows: 6270


,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2022-11-01,1001,Autauga County,Alabama,2022-11-01,3.442677,1463
1,1001_2022-12-01,1001,Autauga County,Alabama,2022-12-01,3.470915,1475
2,1003_2022-11-01,1003,Baldwin County,Alabama,2022-11-01,8.257636,14145
3,1003_2022-12-01,1003,Baldwin County,Alabama,2022-12-01,8.250630,14133
4,1005_2022-11-01,1005,Barbour County,Alabama,2022-11-01,1.247223,247


## Train Dataset preperation

In [6]:
#lets filter out columns
train_raw_df = train.loc[:,['cfips','first_day_of_month','microbusiness_density']]

#lets get a list of unique county ID's
cfips_list = train_raw_df.cfips.unique().tolist()

#print data types of train data (filtered)
#print(train_raw_df.dtypes)

#converting first_day_of_month to datetime object.
train_raw_df['date'] = pd.to_datetime(train_raw_df['first_day_of_month'])

#creating pivot table and converting cfips into columns and date as index.
train_df_pivot = pd.pivot_table(train_raw_df, values='microbusiness_density', index='date', columns='cfips')

train_df_pivot.head()


cfips,1001,1003,1005,1007,1009,1011,1013,1015,1017,1019,...,56027,56029,56031,56033,56035,56037,56039,56041,56043,56045
date,,,,,,,,,,,,,,,,,,,,,
2019-08-01,3.007682,7.239156,1.073138,1.310777,1.544148,0.926372,0.674981,2.691544,1.486542,1.104277,...,1.150575,5.916088,4.062681,30.183279,6.115724,2.640386,26.433363,4.612016,2.603246,1.548594
2019-09-01,2.884870,7.290936,0.995794,1.305176,1.575892,0.998556,0.739265,2.703752,1.355156,1.089747,...,1.150575,5.894528,3.888567,31.712736,6.142085,2.858371,26.611856,4.338306,2.683593,1.530794
2019-10-01,3.055843,7.425439,1.160149,1.254761,1.546415,0.974495,0.713551,2.693764,1.355156,1.084903,...,1.150575,5.881592,3.903076,31.990431,6.181626,2.913635,26.736261,4.420419,2.635385,1.530794
2019-11-01,2.993233,7.426071,1.000628,1.254761,1.573625,1.022618,1.054256,2.739270,1.362664,1.084903,...,1.200600,5.954897,3.845038,32.776520,6.234348,3.008812,26.920164,4.372520,2.683593,1.601994
2019-12-01,2.993233,7.470274,1.000628,1.265965,1.555485,1.034649,1.054256,2.729281,1.343894,1.089747,...,1.200600,5.911776,3.845038,32.806427,6.273890,3.088637,26.903938,4.310935,2.635385,1.584194


#Preparing Test DF

In [7]:
test['date'] = pd.to_datetime(test['first_day_of_month'])

revealed_test['date'] = pd.to_datetime(revealed_test['first_day_of_month'])
revealed_test

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,date
0,1001_2022-11-01,1001,Autauga County,Alabama,2022-11-01,3.442677,1463,2022-11-01
1,1001_2022-12-01,1001,Autauga County,Alabama,2022-12-01,3.470915,1475,2022-12-01
2,1003_2022-11-01,1003,Baldwin County,Alabama,2022-11-01,8.257636,14145,2022-11-01
3,1003_2022-12-01,1003,Baldwin County,Alabama,2022-12-01,8.250630,14133,2022-12-01
4,1005_2022-11-01,1005,Barbour County,Alabama,2022-11-01,1.247223,247,2022-11-01
...,...,...,...,...,...,...,...,...
6265,56041_2022-12-01,56041,Uinta County,Wyoming,2022-12-01,4.050703,588,2022-12-01
6266,56043_2022-11-01,56043,Washakie County,Wyoming,2022-11-01,3.143093,190,2022-11-01
6267,56043_2022-12-01,56043,Washakie County,Wyoming,2022-12-01,3.060380,185,2022-12-01
6268,56045_2022-11-01,56045,Weston County,Wyoming,2022-11-01,1.785395,100,2022-11-01


# Arima Forecasting

In [8]:
counter = 0
for county in tqdm(train_df_pivot.columns):
    county_ser = train_df_pivot[county]
    last_val = county_ser.reset_index()[county][38]
    try:
        # lets convert the values using log differencing for stationarity.
        log_diff_ser = np.log(county_ser).diff().dropna()
    
        #fitting and creating arima model
        model = sm.tsa.ARIMA(log_diff_ser, order=(1,1,1))
        model_fit = model.fit()
    
        #forecasting using arima model
        forecast = np.exp(model_fit.forecast(steps=8).cumsum())*last_val
    
        #adding cfips county code to as a column and removing date as index
        pre_merge_df = forecast.reset_index()
        pre_merge_df['cfips'] = county
        
    except:
        print(county)
        pre_merge_df = pd.DataFrame()
        pre_merge_df['index'] = test['date'].unique()
        pre_merge_df['predicted_mean'] = last_val
        pre_merge_df['cfips'] = county
        
#concatenating the premerge df to final df
    if counter > 0:
        final_forecast_df = pd.concat([final_forecast_df, pre_merge_df])
    else:
        final_forecast_df = copy.deepcopy(pre_merge_df)
        
    counter +=1
    
final_forecast_df

  0%|          | 0/3135 [00:00<?, ?it/s]

28055
48301


,index,predicted_mean,cfips
0,2022-11-01,3.476455,1001
1,2022-12-01,3.490230,1001
2,2023-01-01,3.503913,1001
3,2023-02-01,3.517668,1001
4,2023-03-01,3.531475,1001
...,...,...,...
3,2023-02-01,1.812284,56045
4,2023-03-01,1.819078,56045
5,2023-04-01,1.825898,56045
6,2023-05-01,1.832743,56045


Submission CSV

In [9]:
merged_test_df = pd.merge(test, final_forecast_df, how='inner', left_on=['cfips','date'], right_on=['cfips','index'])
merged_test_df['microbusiness_density'] = merged_test_df['predicted_mean']
submission_df = merged_test_df.loc[:,['row_id','microbusiness_density']]
submission_df.columns = ['row_id','microbusiness_density']
submission_df.to_csv('submission.csv', index=False)
submission_df

,row_id,microbusiness_density
0,1001_2022-11-01,3.476455
1,1003_2022-11-01,8.390583
2,1005_2022-11-01,1.227318
3,1007_2022-11-01,1.290329
4,1009_2022-11-01,1.844942
...,...,...
25075,56037_2023-06-01,2.821838
25076,56039_2023-06-01,26.511083
25077,56041_2023-06-01,4.114288
25078,56043_2023-06-01,3.233005


In [10]:
train_df_pivot[28055]

date
2019-08-01    0.179372
2019-09-01    0.179372
2019-10-01    0.179372
2019-11-01    0.179372
2019-12-01    0.179372
2020-01-01    0.177305
2020-02-01    0.177305
2020-03-01    0.177305
2020-04-01    0.177305
2020-05-01    0.177305
2020-06-01    0.088652
2020-07-01    0.088652
2020-08-01    0.088652
2020-09-01    0.088652
2020-10-01    0.088652
2020-11-01    0.088652
2020-12-01    0.088652
2021-01-01    0.086059
2021-02-01    0.000000
2021-03-01    0.000000
2021-04-01    0.000000
2021-05-01    0.000000
2021-06-01    0.000000
2021-07-01    0.000000
2021-08-01    0.000000
2021-09-01    0.000000
2021-10-01    0.000000
2021-11-01    0.000000
2021-12-01    0.000000
2022-01-01    0.000000
2022-02-01    0.000000
2022-03-01    0.000000
2022-04-01    0.000000
2022-05-01    0.000000
2022-06-01    0.000000
2022-07-01    0.000000
2022-08-01    0.000000
2022-09-01    0.000000
2022-10-01    0.000000
Name: 28055, dtype: float64

In [11]:
revealed_test[revealed_test['cfips']==28055]

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,date
2852,28055_2022-11-01,28055,Issaquena County,Mississippi,2022-11-01,0.000000,0,2022-11-01
2853,28055_2022-12-01,28055,Issaquena County,Mississippi,2022-12-01,0.094607,1,2022-12-01
